# Huang-yi Lee Homework 1: COVID-19 Cases Prediction (Regression)

In [67]:
# Import packages
import math
import argparse
import numpy as np

import os
import csv
import pandas as pd

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

from torch.utils.tensorboard import SummaryWriter

In [68]:
# Some utility functions
def same_seed(seed):
    # Fix random number generator seeds for reproducibility
    torch.backends.mps.deterministic = True
    torch.backends.mps.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)


def train_valid_split(data_set, valid_ratio, seed):
    # Split provided training data into training set and validation set
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)
    

def predict(test_loader, model, device):
    model.eval()
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    return preds

The dataset can be downloaded from https://www.kaggle.com/t/a3ebd5b5542f0f55e828d4f00de8e59a.

In [69]:
# Dataset
class COVID19Dataset(Dataset):
    """
    x: features
    y: targets, if none, do prediction
    """
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
    
    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [70]:
# Neural network model
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
        )
        
    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

In [71]:
# Feature selection
def select_feat(train_data, valid_data, test_data, select_all=True):
    # Select useful features to perform regression
    y_train, y_valid = train_data[:, -1], valid_data[:, -1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], valid_data[:, :-1], test_data
    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_inx = [0, 1, 2, 3, 4]
    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid

In [72]:
# Training loop
def trainer(train_loader, valid_loader, model, opt, device):
    criterion = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(), lr=opt.learning_rate, momentum=opt.momentum)
    writer = SummaryWriter()
    os.makedirs('./models', exist_ok=True)
    n_epochs = opt.n_epochs
    best_loss = math.inf
    step = 0
    early_stop_count = 0
    
    for epoch in range(n_epochs):
        model.train()  # train mode
        loss_record = []
        
        # tqdm is a package to visualize the training progress
        train_pbar = tqdm(train_loader, position=0, leave=True)
        
        for x, y in train_pbar:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            step += 1
            loss_record.append(loss.detach().item())
            
            train_pbar.set_description(f'Epoch [{epoch + 1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
            
        mean_train_loss = sum(loss_record) / len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)
        
        model.eval()  # evaluation mode
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)
            loss_record.append(loss.item())
        
        mean_valid_loss = sum(loss_record) / len(loss_record)
        print(f'Epoch [{epoch + 1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)
        
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), opt.save_path)
            print('Saving model with loss {:3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1
        
        if early_stop_count >= opt.early_stop:
            print('\nModel is not improving, so we halt the training session.')
            return

In [73]:
# Configurations
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
 
opt = argparse.Namespace()
opt.seed = 3
opt.select_all = True
opt.valid_ratio = 0.2
opt.n_epochs = 3000
opt.batch_size = 256
opt.learning_rate = 1e-5
opt.momentum = 0.9
opt.early_stop = 400
opt.save_path = './models/model.ckpt'

In [74]:
# Data loader
same_seed(opt.seed)

train_data, test_data = pd.read_csv('./covid.train.csv').values, pd.read_csv('./covid.test.csv').values
train_data, valid_data = train_valid_split(train_data, opt.valid_ratio, opt.seed)
print(f"""
    train_data size: {train_data.shape}
    valid_data size: {valid_data.shape}
    test_data size: {test_data.shape}
""")

x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, opt.select_all)
print(f'The number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), COVID19Dataset(x_valid, y_valid), COVID19Dataset(x_test)
train_loader = DataLoader(train_dataset, batch_size=opt.batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=opt.batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=opt.batch_size, shuffle=False, pin_memory=True)


    train_data size: (2160, 118)
    valid_data size: (539, 118)
    test_data size: (1078, 117)

The number of features: 117


In [75]:
# Start training!
model = My_Model(input_dim=x_train.shape[1]).to(device)
trainer(train_loader, valid_loader, model, opt, device)

Epoch [1/3000]: 100%|█████████████████| 9/9 [00:00<00:00, 78.55it/s, loss=153]


Epoch [1/3000]: Train loss: 263.3175, Valid loss: 122.5191
Saving model with loss 122.519068...


Epoch [2/3000]: 100%|████████████████| 9/9 [00:00<00:00, 124.73it/s, loss=137]


Epoch [2/3000]: Train loss: 131.9160, Valid loss: 123.4869


Epoch [3/3000]: 100%|████████████████| 9/9 [00:00<00:00, 131.80it/s, loss=127]


...

Epoch [2998/3000]: Train loss: 1.6270, Valid loss: 2.0082


Epoch [2999/3000]: 100%|████████████| 9/9 [00:00<00:00, 136.78it/s, loss=1.06]


Epoch [2999/3000]: Train loss: 1.6088, Valid loss: 1.7903


Epoch [3000/3000]: 100%|█████████████| 9/9 [00:00<00:00, 139.71it/s, loss=2.1]

Epoch [3000/3000]: Train loss: 1.6480, Valid loss: 1.7330


In [78]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(opt.save_path))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv') 

100%|██████████████████████████████████████████| 5/5 [00:00<00:00, 116.28it/s]
